# Generate training data

256×169 samples</p>
### Change demand
timestep = 1h, 1 week ->169 timesteps in total</p>
at every timestep, change the nodal demand</p>
### Change topology
Randomly cut the connectivity between two pipes, but preserving the connectivity between each node in the network and source. ->256 topology

### Try single timestep

In [1]:
import wntr
import networkx as nx
import scipy.sparse as sp
import numpy as np 
import random
import tqdm
import os 
import matplotlib.pyplot as plt
import itertools
from collections import Counter 
import networkx as nx
import copy
import pandas as pd
import tensorflow_gnn as tfgnn
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [2]:
def get_removable_links(inp_file):
    wn = wntr.network.WaterNetworkModel(inp_file)
    removable_links = []
    G = wn.to_graph(wn)
    # Store not-deadend pipes to be removed
    for _, link in wn.links():
        if (link.link_type == 'Pipe' and
            link.start_node.node_type == 'Junction' and
            link.end_node.node_type == 'Junction' and
            G.degree[link.start_node.name] > 1 and
            G.degree[link.end_node.name] > 1):
            removable_links.append(link)
    # To find pairs of links that can be removed together while keeping the network graph connected
    removable_pairs = []
    for (link1, link2) in itertools.combinations(removable_links, 2):
        wnr = copy.deepcopy(wn)  #create a new reference to the same object
        wnr.remove_link(link1)
        wnr.remove_link(link2)
        Gr = wnr.to_graph().to_undirected()
        if nx.is_connected(Gr):
            removable_pairs.append((link1, link2))

    return removable_links, removable_pairs


In [3]:
network = 'ASnet2'
inp_file = network + '.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

# get removable links
removable_links, removable_pairs = get_removable_links(inp_file)
num_nodes = wn.num_nodes
num_links = wn.num_links
num_time = 5
#num_time = int(wn.options.time.duration / wn.options.time.report_timestep + 1)

In [4]:
n_sims = 256
#𝑉_𝑖=[ 𝐼_𝑖^𝑑, 𝑞_𝑖, 𝐼_𝑖^𝑚,𝐻_𝑖^∗ ] 
#𝐸_𝑝=[𝑖,𝑗,𝑐_𝑝]
#Create data matrix
A = np.zeros((n_sims , num_links*num_time, 3))
B = np.zeros((n_sims , num_nodes*num_time, 6))


#A = np.zeros((n_sims * num_time, num_links, 3))
#B = np.zeros((n_sims * num_time, num_nodes, 4))
#U = np.zeros((n_sims * num_time, num_nodes, 1))

# Store the randomly chosen pairs of removable links
links2remove = random.sample(removable_pairs, n_sims)

## Measurement when fully-supervised
measurement_fully = []


In [5]:
for sim in range(n_sims):
    wn = wntr.network.WaterNetworkModel(inp_file)  # reset value
    wn.options.hydraulic.demand_model = 'DD' #dynamic demand model

    i = 0
    for _, node in wn.nodes():
        node.id = i
        i += 1

    if sim != 0:
        (link1, link2) = links2remove[sim - 1]
        wn.remove_link(link1)
        wn.remove_link(link2)

    i = 0
    for _, link in wn.links():
        A[sim, i, 0] = link.start_node.id
        A[sim, i, 1] = link.end_node.id
        A[sim, i, 2] = 1 / (10.667 *link.length / link.roughness ** 1.852 /link.diameter ** 4.871)
        i += 1

    results = wntr.sim.EpanetSimulator(wn).run_sim(version=2.0)
    head = results.node['head']
    demand = results.node['demand']
    demand = np.maximum(demand, 0)
    
    ### Produce datas for multiple timestep
    #### Time step
    index_values = head.index.values
    np.random.seed(42)
    Time_step = np.random.choice(index_values, size=5, replace=False)
    
    repeated_timestep = pd.Series(Time_step).repeat(num_nodes).reset_index(drop=True)
    Time_indicator = pd.DataFrame({'Timestep': repeated_timestep})
    Time_indicator = Time_indicator.squeeze()
    
    #### Demand
    demand_s = demand.loc[Time_step]
    demand_s = demand_s.values.flatten()
    #### Head
    head_s = head.loc[Time_step]
    head_s = head_s.values.flatten()
    #### Node indicator (the number of the node)
    Node_indicator = np.tile(np.arange(1, num_nodes+1), num_time)
        # Measurement when fully-supervised
    measurement_fully = Node_indicator
    #### Junction indicator (if the node is not a reservoir, junction indicator = 1)
    Nd_single = np.array([1 if node.node_type == 'Junction' else 0 for _, node in wn.nodes()])
    Nd = np.tile(Nd_single,num_time)
    Nd = Nd.squeeze()
    #### Measurement indicator (if the node has head, measurement indicator = 1) (fully-supervised, all 1)
    Nh_single = np.zeros(num_nodes)
    Nh = np.tile(Nh_single,num_time)
    Nh = Nh.squeeze()
    ###
    
    ### Node
    B[sim, :, 0] = Time_indicator
    B[sim, :, 1] = Node_indicator
    B[sim, :, 2] = Nd
    B[sim, :, 3] = demand_s
    B[sim, :, 4] = Nh
    B[sim, :, 5] = (1 - Nh) * head_s
    ###


In [6]:
### Edge
A_re = A[:, :65, :]  # Extract the first 65 rows along the second axis

# Repeat the 65 rows to cover the entire second axis
A_re = np.tile(A_re, (1, num_time, 1))


In [12]:
edge_df.head(70)

,source,target,loss_co
0,0.0,1.0,0.044470
1,1.0,2.0,0.058681
2,2.0,3.0,0.031799
3,0.0,4.0,0.084957
4,2.0,48.0,0.033094
...,...,...,...
65,0.0,1.0,0.044470
66,1.0,2.0,0.058681
67,2.0,3.0,0.031799
68,0.0,4.0,0.084957


### Change 3-d array to DataFrame

In [7]:
array_3d_shape = B.shape
new_shape = (array_3d_shape[0]* array_3d_shape[1] ,array_3d_shape[2])

# Reshaping the 3-dimensional array into a 2-dimensional array
B_2d = B.reshape(new_shape)

# Converting the 2-dimensional array into a DataFrame
node_df = pd.DataFrame(B_2d)

array_3d_shape = A_re.shape
new_shape = (array_3d_shape[0]* array_3d_shape[1] ,array_3d_shape[2])

# Reshaping the 3-dimensional array into a 2-dimensional array
A_2d = A_re.reshape(new_shape)

# Converting the 2-dimensional array into a DataFrame
edge_df = pd.DataFrame(A_2d)

In [8]:
edge_df.rename(columns={0: 'source', 1: 'target', 2: 'loss_co'},inplace=True)
node_df.rename(columns={0:'Time_indicator', 1: 'Node_indicator', 2:'Junction_in',3: 'demand', 4: 'Measurement_in',5:'measurement'},inplace=True)

In [10]:
test_edge_df = edge_df[n_sims*num_links*(num_time-1):]
test_node_df = node_df[n_sims*num_nodes*(num_time-1):]

train_edge_df = edge_df[:n_sims*num_links*(num_time-1)]
train_node_df = node_df[:n_sims*num_nodes*(num_time-1)]


In [11]:
def create_graph_tensor(node_df, edge_df):
    graph_tensor = tfgnn.GraphTensor.from_pieces(
        node_sets={
            "node": tfgnn.NodeSet.from_fields(
                sizes=[len(node_df)],
                features={
                    #'Node_indicator':np.array(node_df['Node_indicator'], dtype='int32').reshape(len(node_df),1),
                    'Junction_in':np.array(node_df['Junction_in'], dtype='int32').reshape(len(node_df),1),
                    'demand': np.array(node_df['demand'], dtype='float32').reshape(len(node_df),1),
                    'Measurement_in': np.array(node_df['Measurement_in'], dtype='int32').reshape(len(node_df),1),
                    'measurement': np.array(node_df['measurement'], dtype='float32').reshape(len(node_df),1),
                }
            )
        },
        edge_sets={
            "link": tfgnn.EdgeSet.from_fields(
                sizes=[len(edge_df)],
                features={
                    'loss_co': np.array(edge_df['loss_co'], dtype='float32').reshape(len(edge_df),1),
                },
                adjacency=tfgnn.Adjacency.from_indices(
                                          source=("node", np.array(edge_df['source'], dtype='int32')),
                                          target=("node", np.array(edge_df['target'], dtype='int32')),
                                      ))
        }
    )

    return graph_tensor


In [12]:
full_tensor = create_graph_tensor(node_df, edge_df)
train_tensor = create_graph_tensor(train_node_df, train_edge_df)
test_tensor = create_graph_tensor(test_node_df, test_edge_df)

split off ‘head’ as our target (label).

In [13]:
def node_batch_merge(graph):
    graph = graph.merge_batch_to_components()
    node_features = graph.node_sets['node'].get_features_dict()
    edge_features = graph.edge_sets['link'].get_features_dict()

    label = node_features.pop('measurement')
    print(label)

    new_graph = graph.replace_features(node_sets={'node': node_features}, edge_sets={'link': edge_features})
    return new_graph, label


def create_dataset(graph, function):
    dataset = tf.data.Dataset.from_tensors(graph)
    dataset = dataset.batch(32)
    return dataset.map(function)

### Train dataset

In [14]:
full_node_dataset = create_dataset(full_tensor, node_batch_merge)
train_node_dataset = create_dataset(train_tensor, node_batch_merge)
test_node_dataset = create_dataset(test_tensor, node_batch_merge)

Tensor("Reshape_5:0", shape=(None, 1), dtype=float32)
Tensor("Reshape_5:0", shape=(None, 1), dtype=float32)
Tensor("Reshape_5:0", shape=(None, 1), dtype=float32)


# Model Building

## Build model inputs

In [15]:
graph_spec = train_node_dataset.element_spec[0]
input_graph = tf.keras.layers.Input(type_spec=graph_spec)

In [18]:
def set_initial_node_state(node_set, node_set_name):
    features = [
        #tf.keras.layers.Dense(32, activation="relu")(node_set['Node_indicator']),
        tf.keras.layers.Dense(32, activation="relu")(node_set['Junction_in']),
        tf.keras.layers.Dense(32, activation="relu")(node_set['demand']),
        tf.keras.layers.Dense(32, activation="relu")(node_set['Measurement_in']),
    ]
    return tf.keras.layers.Concatenate()(features)


def set_initial_edge_state(edge_set, edge_set_name):
    features = [
        tf.keras.layers.Dense(32, activation="relu")(edge_set['loss_co']),
    ]
    return tf.keras.layers.Concatenate()(features)




In [19]:
graph = tfgnn.keras.layers.MapFeatures(node_sets_fn=set_initial_node_state,
                                       edge_sets_fn=set_initial_edge_state)(input_graph)

In [20]:
def dense_layer(units=32, l2_reg=0.1, dropout=0.25, activation='relu'):
    regularizer = tf.keras.regularizers.l2(l2_reg)
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units, kernel_regularizer=regularizer, bias_regularizer=regularizer),
        tf.keras.layers.Dropout(dropout)
    ])


### Compile

In [21]:
graph_updates = 3
for i in range(graph_updates):
    graph = tfgnn.keras.layers.GraphUpdate(
        node_sets={
            'node':
            tfgnn.keras.layers.NodeSetUpdate(
                {
                    'link':
                    tfgnn.keras.layers.SimpleConv(message_fn=dense_layer(32),
                                                  reduce_type="sum",
                                                  sender_edge_feature=tfgnn.HIDDEN_STATE,
                                                  receiver_tag=tfgnn.TARGET)
                }, tfgnn.keras.layers.NextStateFromConcat(dense_layer(64)))
        })(graph)


logits = tf.keras.layers.Dense(1)(graph.node_sets["node"][tfgnn.HIDDEN_STATE])

node_model = tf.keras.Model(input_graph, logits)
node_model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
                   loss='mean_squared_error',
                   metrics=['mean_squared_error'])
node_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [()]                      0         
                                                                 
 map_features_1 (MapFeatures  ()                       256       
 )                                                               
                                                                 
 graph_update (GraphUpdate)  ()                        15456     
                                                                 
 graph_update_1 (GraphUpdate  ()                       11360     
 )                                                               
                                                                 
 graph_update_2 (GraphUpdate  ()                       11360     
 )                                                               
                                                             

### Train model

In [22]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50, restore_best_weights=True)

node_model.fit(train_node_dataset.repeat(),
               validation_data=full_node_dataset,
               steps_per_epoch=10,
               epochs=1000,
               callbacks=[es])

Epoch 1/1000
10/10 [==============================] - 5s 304ms/step - loss: 1317883871232000.0000 - mean_squared_error: 1317884005449728.0000 - val_loss: 140075098701824.0000 - val_mean_squared_error: 140075098701824.0000
Epoch 2/1000
10/10 [==============================] - 3s 280ms/step - loss: 600725600075776.0000 - mean_squared_error: 600725600075776.0000 - val_loss: 70389489205248.0000 - val_mean_squared_error: 70389489205248.0000
Epoch 3/1000
10/10 [==============================] - 3s 279ms/step - loss: 160270999617536.0000 - mean_squared_error: 160270999617536.0000 - val_loss: 11942175440896.0000 - val_mean_squared_error: 11942175440896.0000
Epoch 4/1000
10/10 [==============================] - 3s 278ms/step - loss: 31712916537344.0000 - mean_squared_error: 31712916537344.0000 - val_loss: 10444839321600.0000 - val_mean_squared_error: 10444839321600.0000
Epoch 5/1000
10/10 [==============================] - 3s 311ms/step - loss: 9462168420352.0000 - mean_squared_error: 946216842

10/10 [==============================] - 4s 429ms/step - loss: 270177091584.0000 - mean_squared_error: 270177091584.0000 - val_loss: 3570403584.0000 - val_mean_squared_error: 3570403584.0000
Epoch 41/1000
10/10 [==============================] - 4s 411ms/step - loss: 141449691136.0000 - mean_squared_error: 141449691136.0000 - val_loss: 18311479296.0000 - val_mean_squared_error: 18311479296.0000
Epoch 42/1000
10/10 [==============================] - 4s 391ms/step - loss: 169649717248.0000 - mean_squared_error: 169649717248.0000 - val_loss: 805395904.0000 - val_mean_squared_error: 805395904.0000
Epoch 43/1000
10/10 [==============================] - 4s 401ms/step - loss: 157640425472.0000 - mean_squared_error: 157640425472.0000 - val_loss: 3913114880.0000 - val_mean_squared_error: 3913114880.0000
Epoch 44/1000
10/10 [==============================] - 4s 426ms/step - loss: 154708312064.0000 - mean_squared_error: 154708312064.0000 - val_loss: 1288013312.0000 - val_mean_squared_error: 12880

10/10 [==============================] - 5s 446ms/step - loss: 80969760768.0000 - mean_squared_error: 80969760768.0000 - val_loss: 3699984640.0000 - val_mean_squared_error: 3699984640.0000
Epoch 81/1000
10/10 [==============================] - 4s 445ms/step - loss: 66899025920.0000 - mean_squared_error: 66899025920.0000 - val_loss: 545622208.0000 - val_mean_squared_error: 545622208.0000
Epoch 82/1000
10/10 [==============================] - 4s 425ms/step - loss: 48355504128.0000 - mean_squared_error: 48355500032.0000 - val_loss: 7794520064.0000 - val_mean_squared_error: 7794520064.0000
Epoch 83/1000
10/10 [==============================] - 4s 430ms/step - loss: 39639506944.0000 - mean_squared_error: 39639506944.0000 - val_loss: 1932137472.0000 - val_mean_squared_error: 1932137472.0000
Epoch 84/1000
10/10 [==============================] - 4s 405ms/step - loss: 34639355904.0000 - mean_squared_error: 34639355904.0000 - val_loss: 2998423808.0000 - val_mean_squared_error: 2998423808.0000
E

KeyboardInterrupt: 

### Evaluate model

In [86]:
eval_result = node_model.evaluate(test_node_dataset)
print(dict(zip(node_model.metrics_names, eval_result)))

1/1 [==============================] - 0s 102ms/step - loss: 4705823744.0000 - mean_squared_error: 4705823744.0000
{'loss': 4705823744.0, 'mean_squared_error': 4705823744.0}
